In [1]:
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import os
import cv2
import torch
import numpy as np
import pandas as pd
import requests
from PIL import Image
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from natsort import natsorted
from tqdm import tqdm_notebook
from tqdm import tqdm
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import json

def encode_image(img , transform):
    input_tensor = transform(img)
    input_tensor = input_tensor.half()
    input_tensor = input_tensor.unsqueeze(0)
    input_tensor = input_tensor.to(device)
    output_tensor = pipe.vae.encode(input_tensor)
    return output_tensor.latent_dist.sample()[0]



def encode_image_avator(img , transform):
    transformed_image = transform(Image.fromarray(np.array(img)[:, :, :3]))
    pil_transformed_image = transforms.ToPILImage()(transformed_image)
    return encode_image(pil_transformed_image, transform)

def calculate_mse(tensor1, tensor2):
    return torch.mean((tensor1 - tensor2) ** 2)

def calculate_pearson_similarity(tensor1, tensor2):
    mean_tensor1 = torch.mean(tensor1)
    mean_tensor2 = torch.mean(tensor2)
    centered_tensor1 = tensor1 - mean_tensor1
    centered_tensor2 = tensor2 - mean_tensor2
    dot_product = torch.sum(centered_tensor1 * centered_tensor2)
    norm_tensor1 = torch.norm(centered_tensor1)
    norm_tensor2 = torch.norm(centered_tensor2)
    similarity = dot_product / (norm_tensor1 * norm_tensor2)
    return similarity

def calculate_cosine_similarity(tensor1, tensor2):
    dot_product = torch.sum(tensor1 * tensor2)
    norm_tensor1 = torch.norm(tensor1)
    norm_tensor2 = torch.norm(tensor2)
    similarity = dot_product / (norm_tensor1 * norm_tensor2)
    return similarity


avator_path = './data/avator/PRIEST'
file_list = os.listdir(avator_path)
folder_list = [item for item in file_list if not '.' in item]

belt_bb = [60, 80, 180, 140]
pants_bb = [50, 120, 190, 210]
shoes_bb = [20, 200, 200, 240]
face_bb = [80, 10, 150, 55]
neck_bb = [100, 30, 150, 70]
cap_bb = [90, 0, 150, 20]
coat_bb = [20, 40, 200, 110]
coat_bb = [20, 40, 200, 110]
hair_bb = [90, 0, 150, 20]

bb_dict = {}



folder_list = ['hair' , 'cap' , 'face' , 'neck' , 'coat' , 'belt' , 'pants' , 'shoes']
position_list = [item + ' position' for item in folder_list]
avator_path = './data/avator/PRIEST'

        
for part, bb in zip(folder_list, [hair_bb, cap_bb, face_bb, neck_bb, coat_bb, belt_bb, pants_bb, shoes_bb]):
    bb_dict[part] = bb

print(bb_dict)


transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image to a desired size
    transforms.ToTensor(),       # Convert the image to a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


{'hair': [90, 0, 150, 20], 'cap': [90, 0, 150, 20], 'face': [80, 10, 150, 55], 'neck': [100, 30, 150, 70], 'coat': [20, 40, 200, 110], 'belt': [60, 80, 180, 140], 'pants': [50, 120, 190, 210], 'shoes': [20, 200, 200, 240]}


In [2]:
device = "cuda"
class ImageHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory and event.src_path.endswith(('.jpg', '.jpeg', '.png')):
            print("New image file created:", event.src_path)
            pil_image = Image.open(event.src_path)
            prompt = "a men wearing game costume"

            target_size = (244, 244)  # Set your desired size
            resized_image = pil_image.resize(target_size)
            
            #images = pipe(prompt=prompt,  image=resized_image, num_inference_steps = 150).images[0]
            images = pipe(prompt=prompt, image=resized_image, guidance_scale=1, strength=0.25).images[0]
            print("Generate image")
            plt.imshow(images)
                        
            recommed_dict = {}

            for part in bb_dict:
                part_img = images.crop((bb_dict[part][0], bb_dict[part][1], bb_dict[part][2], bb_dict[part][3]))
                image_lv = encode_image(part_img , transform)

                part_latents = []
                part_avators = []
                folder_list = ['hair' , 'cap' , 'face' , 'neck' , 'coat' , 'belt' , 'pants' , 'shoes']
                avator_path = './data/avator/PRIEST'

                full_paths = os.path.join(avator_path, part)
                part_imgs = natsorted(os.listdir(full_paths))
                part_imgs = [file for file in part_imgs if file.lower().endswith('.png')]
                for path in tqdm(part_imgs , desc = f'{part}'):
                    
                    part_img_path = os.path.join(f'./data/avator/PRIEST/{part}' , path)
                    #print(part_img_path)
                    avator_part = Image.open(part_img_path)
                    avator_part = cv2.cvtColor(np.array(avator_part), cv2.COLOR_BGR2RGB)
                    part_latents.append(encode_image_avator(avator_part, transform))
                    part_avators.append(avator_part)
                    
                mse_list = [calculate_mse(image_lv, t).item() for t in part_latents]
                part_rec_mse = np.array(mse_list).argmin()
                plt.imshow(part_avators[part_rec_mse])
                recommed_dict[part+' position'] = str(part_rec_mse*28)
            
            json_data = json.dumps([recommed_dict])
            save_name = event.src_path.split('/')[-1]
            print(save_name)

            with open(f'./{output_dir}/{save_name}.json', 'w') as file:
                file.write(json_data)


            print('create json')
            print(f'create json name : ')
            with open(f'./{output_dir}/{save_name}.json', 'rb') as f:
                string_param = save_name.split('.')[0] +"."+ save_name.split('.')[1]
                files = {'file': f }
                data = {'userid': string_param}

                response = requests.post(post_url, data=data, files=files)
            print('Response status code:', response.status_code)
            # Print more detailed response information
            print('Response headers:', response.headers)
            print('Response body:', response.text)

            # If the response is in JSON format, you can use response.json() to parse it
            try:
                print('Parsed JSON response:', response.json())
            except Exception as e:
                print('Could not parse response as JSON. Exception:', str(e))

            os.remove(event.src_path)
            os.remove(f'./{output_dir}/{save_name}.json')
            print('Image file deleted:', event.src_path)



In [ ]:
if __name__ == "__main__":
    path = "input_image"  # 이미지 파일이 생성될 경로
    output_dir = 'result'
    post_url = 'https://956d-125-137-22-250.ngrok-free.app/upload'

    model_path = "./output/pytorch_lora_weights.bin"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        safety_checker=None,
        feature_extractor=None,
        requires_safety_checker=False
    )

    # load lora weights
    pipe.unet.load_attn_procs(model_path)
    # set to use GPU for inference
    pipe.to(device)
    print('load model')

    event_handler = ImageHandler()
    observer = Observer()
    observer.schedule(event_handler, path, recursive=False)
    observer.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()

    observer.join()

load model
New image file created: input_image/zeskywa499@gmail.com.jpg


  0%|          | 0/12 [00:00<?, ?it/s]

Generate image


shoes: 100%|██████████| 1165/1165 [00:53<00:00, 21.67it/s]


zeskywa499@gmail.com.jpg
create json
create json name : 
Response status code: 200
Response headers: {'Content-Length': '6', 'Content-Type': 'text/plain;charset=UTF-8', 'Date': 'Sun, 11 Jun 2023 13:09:59 GMT', 'Ngrok-Trace-Id': 'a5706ade23031edd0e032eb763d82009'}
Response body: 성공
Could not parse response as JSON. Exception: Expecting value: line 1 column 1 (char 0)
Image file deleted: input_image/zeskywa499@gmail.com.jpg
New image file created: input_image/dlwjdgns30111@gmail.com.jpg


...

dlwjdgns30111@gmail.com.jpg
create json
create json name : 
Response status code: 200
Response headers: {'Content-Length': '6', 'Content-Type': 'text/plain;charset=UTF-8', 'Date': 'Sun, 11 Jun 2023 13:21:40 GMT', 'Ngrok-Trace-Id': '5d063153928bd610f2a963fa77a094fa'}
Response body: 성공
Could not parse response as JSON. Exception: Expecting value: line 1 column 1 (char 0)
Image file deleted: input_image/dlwjdgns30111@gmail.com.jpg
New image file created: input_image/dlwjdgns30111@gmail.com.jpg


  0%|          | 0/12 [00:00<?, ?it/s]

Generate image


shoes: 100%|██████████| 1165/1165 [00:17<00:00, 67.89it/s]


dlwjdgns30111@gmail.com.jpg
create json
create json name : 
Response status code: 200
Response headers: {'Content-Length': '6', 'Content-Type': 'text/plain;charset=UTF-8', 'Date': 'Sun, 11 Jun 2023 13:49:12 GMT', 'Ngrok-Trace-Id': '7d614ba2dbd9101e34282eb404ff76c6'}
Response body: 성공
Could not parse response as JSON. Exception: Expecting value: line 1 column 1 (char 0)
Image file deleted: input_image/dlwjdgns30111@gmail.com.jpg
